In [ ]:
import pandas as pd

In [ ]:
data = pd.read_excel(r"CocaCola_Sales_Rawdata.xlsx")
data

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
plt.title("Line Plot", size = 15, weight = 'bold')
plt.ylabel("Sales", size = 10, weight = 'bold')
plt.plot(data['Sales'])

In [ ]:
data.hist()

In [ ]:
data.plot(kind = 'kde')
plt.title("Density Plot",size = 15, weight = 'bold')

In [ ]:
quarters = ['Q1','Q2','Q3','Q4']
n = data['Quarter'][0]
n[0:2]

In [ ]:
data['quarter'] = 0
for i in range(42):
    n = data['Quarter'][i]
    data['quarter'][i] = n[0:2]

In [ ]:
dummy = pd.DataFrame(pd.get_dummies(data['quarter']))
dummy

In [ ]:
data_c = pd.concat([data,dummy], axis = 1)
data_c

In [ ]:
data_c["t"] = np.arange(1,43)

data_c["t_squared"] = data_c["t"]*data_c["t"]
data_c.columns
data_c["log_Sales"] = np.log(data_c["Sales"])
data_c

In [ ]:
plt.figure(figsize=(12,8))
heatmap_y_quarter = pd.pivot_table(data=data_c,values="Sales",index="Quarter",columns="quarter",aggfunc="mean", fill_value = 0)
sns.heatmap(heatmap_y_quarter,annot=True,fmt="g") #fmt is format of the grid values

In [ ]:
plt.figure(figsize=(8,6))
sns.boxplot(x="quarter",y="Sales",data=data_c)

In [ ]:
from pandas.plotting import lag_plot
from statsmodels.graphics.tsaplots import plot_acf

In [ ]:
lag_plot(data_c['Sales'])
plt.title("Lag Plot", size = 15, weight = "bold")
plt.show()

In [ ]:
plot_acf(data_c['Sales'], lags = 30, color = 'darkblue')                                              # lags = 30 means it will plot for k = 30 lags
plt.xlabel("No of lags, k = 30", size = 10, weight = 'bold')
plt.ylabel("Autocorrelation (r2 value)", size = 10, weight = 'bold')
plt.show()

In [ ]:
plt.figure(figsize=(12,3))
sns.lineplot(x="quarter",y="Sales",data=data_c)

In [ ]:
data_c['Sales'].plot()

train = data_c.head(37)
test  = data_c.tail(4)

In [ ]:
train

In [ ]:
test

In [ ]:
import statsmodels.formula.api as smf

linear = smf.ols('Sales~t',data=train).fit()
pred_linear =  pd.Series(linear.predict(pd.DataFrame(test['t'])))
rmse_linear = np.sqrt(np.mean((np.array(test['Sales'])-np.array(pred_linear))**2))
rmse_linear

In [ ]:
Exp = smf.ols('log_Sales~t',data=train).fit()
pred_Exp = pd.Series(Exp.predict(pd.DataFrame(test['t'])))
rmse_Exp = np.sqrt(np.mean((np.array(test['Sales'])-np.array(np.exp(pred_Exp)))**2))
print("Root Mean Square Error : ",rmse_Exp)

In [ ]:
Quad = smf.ols('Sales~t+t_squared',data=train).fit()
pred_Quad = pd.Series(Quad.predict(test[["t","t_squared"]]))
rmse_Quad = np.sqrt(np.mean((np.array(test['Sales'])-np.array(pred_Quad))**2))
print("Root Mean Square Error : ",rmse_Quad)

In [ ]:
add_sea = smf.ols('Sales~Q1+Q2+Q3+Q4',data=train).fit()
pred_add_sea = pd.Series(add_sea.predict(test[['Q1','Q2','Q3','Q4']]))
rmse_add_sea = np.sqrt(np.mean((np.array(test['Sales'])-np.array(pred_add_sea))**2))
print("Root Mean Square Error : ",rmse_add_sea)